In [1]:
from src.produce_datasets import *
from src.model_tester import *
from src.dictionaries import *

In [2]:
    df_2020 = feature_space('2020')
    df_2020 = df_2020[df_2020['10'] != 0]
    for column in features:
        if column not in df_2020.columns:
            df_2020[column] = 0

In [3]:
df = pd.read_json('data/training_dataset.json')

In [4]:
    X = df.loc[:,features].values
    X_prime = df_2020.loc[:,features].values
    y_recovery = df.loc[:,['recovery']].values * 1
    y_recovery = y_recovery.reshape(len(y_recovery,))
    y_delta = df.loc[:,['delta']].values * 1
    y_decline = df.loc[:,['decline']].values * 1    
    X_train, X_test, y_train, y_test = train_test_split(X, y_recovery, test_size=.2, stratify = y_recovery)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_prime = scaler.transform(X_prime)

In [5]:
recovery_model = RFC(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=15, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=10, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, verbose=1,
                        warm_start=False)
recovery_model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.5s finished


RandomForestClassifier(max_depth=15, min_samples_leaf=10, verbose=1)

In [6]:
    recov_predict =  recovery_model.predict(X_prime)                    
    recov_likelihood = recovery_model.predict_proba(X_prime)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [7]:
df_2020['recov_predict'] = recov_predict

In [8]:
df_2020['recov_likelihood'] = recov_likelihood[:,1:]

In [9]:
df_2020.to_json('data/prediction_2020.json')

In [10]:
df = pd.read_json('data/prediction_2020.json')
df

,10,101,1011,1012,1013,102,1021,1022,1023,1024,...,7221,72211,722110,7222,72221,722211,722212,722213,recov_predict,recov_likelihood
01000,1983835,0.193093,0.009052,0.047551,0.136490,0.619646,0.189821,0.010413,0.047987,0.126556,...,0,0,0,0,0,0,0,0,1,0.849120
01001,11070,0.200903,0.016260,0.043812,0.140831,0.564408,0.191509,0.002349,0.038753,0.056188,...,0,0,0,0,0,0,0,0,1,0.645615
01003,76061,0.121771,0.009808,0.057191,0.054772,0.743272,0.232195,0.005864,0.053352,0.082789,...,0,0,0,0,0,0,0,0,1,0.832991
01005,8410,0.350654,0.034245,0.017717,0.298692,0.451605,0.161237,0.001784,0.027586,0.058264,...,0,0,0,0,0,0,0,0,0,0.426021
01007,4868,0.319022,0.014585,0.183648,0.120789,0.411873,0.198233,0.000000,0.020542,0.025267,...,0,0,0,0,0,0,0,0,0,0.451277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CS566,230509,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,0.574787
US000,147088945,0.147475,0.012011,0.049439,0.086025,0.701259,0.184874,0.019602,0.056622,0.142778,...,0,0,0,0,0,0,0,0,1,0.917172
USCMS,112858175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,0.580158
USMSA,127515349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,0.580158
